In [5]:
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from tensorflow.keras.preprocessing.text import Tokenizer

import mlflow

max_words = 1000
batch_size = 32
epochs = 5

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Loading data...
2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
8982 train sequences
2246 test sequences
46 classes
Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [7]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Iron")

2025/11/04 14:05:21 INFO mlflow.tracking.fluent: Experiment with name 'Iron' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/195699277622829892', creation_time=1762261521516, experiment_id='195699277622829892', last_update_time=1762261521516, lifecycle_stage='active', name='Iron', tags={}>

In [8]:
mlflow.tensorflow.autolog()
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

2025/11/04 14:05:22 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '108204e58f894339b74c6380b8125882', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Building model...


Epoch 1/5
248/253 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5868 - loss: 1.8561

253/253 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6798 - loss: 1.4193 - val_accuracy: 0.7597 - val_loss: 1.0717
Epoch 2/5
248/253 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8058 - loss: 0.8170

253/253 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8116 - loss: 0.7831 - val_accuracy: 0.7976 - val_loss: 0.9072
Epoch 3/5
252/253 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8686 - loss: 0.5380

253/253 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8659 - loss: 0.5439 - val_accuracy: 0.8076 - val_loss: 0.8660
Epoch 4/5
242/253 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9074 - loss: 0.4091

253/253 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9000 - loss: 0.4229 - val_accuracy: 0.8120 - val_loss: 0.8558
Epoch 5/5
253/253 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9205 - loss: 0.3258 - val_accuracy: 0.8031 - val_loss: 0.9003
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


2025/11/04 14:05:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run aged-sow-379 at: http://127.0.0.1:5000/#/experiments/195699277622829892/runs/108204e58f894339b74c6380b8125882
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/195699277622829892
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7947 - loss: 0.8667
Test score: 0.8667305111885071
Test accuracy: 0.7947462201118469


In [10]:
mlflow.log_param("batch_size", batch_size)
mlflow.log_param("learning_rate", 0.01)

0.01

In [14]:
mlflow.log_metric("value", score[1])

In [16]:
mlflow.set_tag("project", "reuters_classification")